# Submission details
*   **Name**: Aditya Verma
*  **Student ID**: 12948511
*   **Task**: Object Detection - Assignment 2
*   **Subject**: 42028 Deep Learning and Convolutional Neural Network - Autumn 2020
*   **DataSet**: Red Blood Cell Detection and Counting
*   **Model Used**: 




Drive Mount

In [0]:
#Mounting to google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. Imports

In [0]:
# Checking that the tensforflow version is 1 as object detection will not work on version 2
%tensorflow_version 1.x
# Ensuring numpy version 1.17.4 is installed
!pip install numpy==1.17.4

TensorFlow 1.x selected.


In [0]:
!python -c 'import tensorflow as tf; print(tf.__version__)'

1.15.2


1.15.2

In [0]:
import re
import numpy as np

import os
from glob import glob
import shutil

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import tensorflow.compat.v1 as tf #tensor flow

#2. Data Preperation

##Sets the working directory

In [0]:
# Changing to the working directory to access dataset
%cd /content/drive/My Drive/Assign_2/object-detection/

/content/drive/My Drive/Assign_2/object-detection


##Unzip the original .zip dataset to prerpare for processing

In [0]:
 # Unzips the dataset
 !unzip BloodRBC-Detection-Dataset-Processed.zip

##Assigning and creating directories and variable

###Create Folders required for training

In [0]:
base_dir = '/content/drive/My Drive/Assign_2/object-detection/'
dataset_dir = "/content/drive/My Drive/Assign_2/object-detection/BloodRBC-Detection-Dataset-Processed"

test_dir = "/content/drive/My Drive/Assign_2/object-detection/images/test/"
train_dir = "/content/drive/My Drive/Assign_2/object-detection/images/train/"
data_dir = "/content/drive/My Drive/Assign_2/object-detection/data/"

Confirmation that files have been created or already exist

In [0]:
try: os.mkdir(test_dir)
except OSError: print ("Creation of the directory %s failed" % test_dir)
else: print ("Successfully created the directory %s " % test_dir)
try: os.mkdir(train_dir)
except OSError: print ("Creation of the directory %s failed" % train_dir)
else: print ("Successfully created the directory %s " % train_dir)
try: os.mkdir(data_dir)
except OSError: print ("Creation of the directory %s failed" % data_dir)
else: print ("Successfully created the directory %s " % data_dir)


Creation of the directory /content/drive/My Drive/Assign_2/object-detection/images/test/ failed
Creation of the directory /content/drive/My Drive/Assign_2/object-detection/images/train/ failed
Creation of the directory /content/drive/My Drive/Assign_2/object-detection/data/ failed


##Deletes jpegs which do not have corresponding xml

In [0]:
files = os.listdir(dataset_dir)

count = 0

for file in tqdm(files):
    filename, filetype = file.split('.')

    if filetype == 'xml':
        continue
    imgfile = os.path.join(dataset_dir, file)
    xmlfile = os.path.join(dataset_dir, filename + '.xml')
    if not os.path.exists(xmlfile):
        print('{} deleted.'.format(imgfile))
        os.remove(imgfile)
        count = count + 1
print("Number of jpegs deleted",count)

100%|██████████| 686/686 [00:00<00:00, 11426.03it/s]

Number of jpegs deleted 0


Expected result: *Number of jpegs with corresponding xml is:  343.0*

In [0]:
print('Number of jpegs with corresponding xml is: ', len(os.listdir(dataset_dir))/2)

Number of jpegs with corresponding xml is:  343.0


#MANUALLY move Images and corresponding xml
I have manually moved the images and xml files into seperate folders.
The split format is roughly 90% Training to 10% Test(51 Images).
Images and xml ranging from 00000 to 00351 have been moved to /images/train. And the rest have been moved to /images/test/.

#Generating tfrecords


In [0]:
# We need to downloadthe tensorflow models from github
!git clone https://github.com/tensorflow/models.git

In [0]:
# This will be converting the xml's into csv
!python3 xml_to_csv.py

In [0]:
%cd /content/drive/My Drive/Assign_2/object-detection/models/research
!python3 setup.py install

In [0]:
%cd /content/drive/My Drive/Assign_2/object-detection/

/content/drive/My Drive/Assign_2/object-detection


In [0]:
# Generating tf records for the train labels 
# using the python file within the tensorflow model directory
!python3 generate_tfrecord.py --csv_input=data/train_labels.csv --output_path=data/train.record --image_dir=images/train
# Generating tf records for the test labels 
# using the python file within the tensorflow model directory and then saving to 
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv --output_path=data/test.record --image_dir=images/test

#Configuration of SSD Model

##Downloading files

###Downloading the config file from github and Manually changing the values
Please note: 

In [0]:
!wget hhttps://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/samples/configs/ssd_mobilenet_v1_pets.config

After downloading the number of classes and paths need to be edited in this file.


*   Number of classes needs to be changed to 1
*   Directories need to be assigned to base folder



###Downloading the SSD model

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz

###Unzips the tar.gz

In [0]:
!tar xvzf ssd_mobilenet_v1_coco_11_06_2017.tar.gz

##Creating the label-map
The following code will be added to object-detector.pbtxt file within the training folder

item {
  id: 1
  name: 'RBC'
}

##Moving files into the model folder


In [0]:
# The files and folders created will need to be moved into the models directory so that the object detection can process
!mv data/test_labels.csv models/research/object_detection/data
!mv data/train_labels.csv models/research/object_detection/data
!mv data/test.record models/research/object_detection/data
!mv data/train.record models/research/object_detection/data


!mv images models/research/object_detection/
!mv ssd_mobilenet_v1_coco_11_06_2017 models/research/object_detection/
!mv training models/research/object_detection/
!mv ssd_mobilenet_v1_pets.config models/research/object_detection/training/

models\research\slim\nets and models\research\slim\deployment need to be moved into models\research\object_detection\legacy\ to avoid the "No module named 'nets'" error

##Running the model

In [0]:
%cd models/research/object_detection/legacy/

/content/drive/My Drive/Assign_2/object-detection/models/research/object_detection/legacy


In [0]:
!dir

In [0]:
!pip install tensorflow-object-detection-api

In [0]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [0]:
!python3 train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config

#TensorBoard to visualize progress of training process

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-05-21 00:51:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.229.170.137, 54.164.74.108, 3.90.94.177, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.229.170.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  51.6MB/s    in 0.3s    

2020-05-21 00:51:04 (51.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
model_dir = '/training'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')


In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://659bd026.ngrok.io


##Frozen model for inference

In [0]:
!tensorboard --logdir=summaries

TensorBoard 1.15.0 at http://70ef70561862:6006/ (Press CTRL+C to quit)
^C


#Faster R-CNN